In [1]:
import pandas as pd

df = pd.read_csv('../../Downloads/train.csv')
df.head()

,ChannelName,ChannelId,MessageId,Date,EditDate,Content,Suspicious_Level
0,boris_rozhin,1101806611,91626,2023-07-08 16:11:34,2023-07-08 16:11:47,Работа наших бойцов к югу от Артемовска. Работ...,2
1,sashakots,1109403194,40853,2023-07-08 16:44:44,2023-07-08 16:44:58,"Анкара нарушила договорённости, отпустив глава...",1
2,swodki,1144180066,280668,2023-07-09 02:00:23,2023-07-09 02:05:53,ЭТО ЕДИНСТВЕННЫЙ СПОСОБ ПОМОЧЬ НАМ! \n\nПополн...,1
3,boris_rozhin,1101806611,91573,2023-07-08 02:07:05,2023-07-08 02:07:19,МТ-ЛБ с 32-зарядной авиационной пусковой устан...,1
4,swodki,1144180066,280695,2023-07-09 07:01:49,2023-07-09 07:05:08,ЭТО ЕДИНСТВЕННЫЙ СПОСОБ ПОМОЧЬ НАМ! \n\nПополн...,1


In [2]:
df['binary_target'] = (df.Suspicious_Level > 2).astype(int)

In [3]:
from sklearn.model_selection import train_test_split

df_train, df_test = train_test_split(df, test_size=0.2, random_state=42, stratify=df.binary_target)

In [4]:
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB, GaussianNB
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.pipeline import Pipeline
from sklearn.metrics import roc_auc_score

nb_clf = Pipeline([
        ('vectorizer', CountVectorizer(
            analyzer='word',
            ngram_range=(1, 3),
            lowercase=True,
            min_df=5,
            max_features=30000,
            binary=True
        )),  
        ('classifier', MultinomialNB())
    ])

lr_clf = Pipeline([
        ('vectorizer', CountVectorizer(
            analyzer='word',
            ngram_range=(1, 3),
            lowercase=True,
            min_df=5,
            max_features=30000,
            binary=True
        )),  
        ('classifier', LogisticRegression(C=1.0, random_state=42))
    ])

In [5]:
nb_clf.fit(df_train.Content, df_train.binary_target)

roc_auc_score(df_test.binary_target, nb_clf.predict_proba(df_test.Content)[:, 1])

0.8912087912087912

In [6]:
lr_clf.fit(df_train.Content, df_train.binary_target)

roc_auc_score(df_test.binary_target, lr_clf.predict_proba(df_test.Content)[:, 1])

0.8234432234432235

In [7]:
lr_clf = Pipeline([
        ('vectorizer', TfidfVectorizer(
            analyzer='word',
            ngram_range=(1, 3),
            lowercase=True,
            min_df=5,
            max_features=30000,
            binary=True
        )),  
        ('classifier', LogisticRegression(C=0.1, random_state=42))
    ])

In [8]:
lr_clf.fit(df_train.Content, df_train.binary_target)

roc_auc_score(df_test.binary_target, lr_clf.predict_proba(df_test.Content)[:, 1])

0.8845054945054945